# UCB-VIアルゴリズムのリグレットの証明

参考
* [Reinforcement Learning: Theory and Algorithms](https://rltheorybook.github.io/)
* [Covering with $\varepsilon$-nets](https://erdogdu.github.io/csc2532/lectures/lecture05.pdf)
* [Notes on Exploration in Linear MDPs](https://nanjiang.cs.illinois.edu/files/cs542f22/note8.pdf)
* [Useful Inequalities Cheat Sheet](https://www.lkozma.net/inequalities_cheat_sheet/ineq.pdf)
* [Upper Bound of $\sum_{k=1}^{n} \frac{1}{\sqrt{k}}$](https://math.stackexchange.com/questions/3745324/upper-bound-of-sum-k-1n-frac1-sqrtk)

[前回](UCB_regret_proof.ipynb)の拡張として、RLにおけるUCB-VIアルゴリズムの偽リグレットを簡潔に証明します。
期待リグレットは今回の導出において確率$\delta$以下で成立するBadイベントを考えます。
面倒なので定数係数は全部$\square$でまとめます。

## Tabular MDP

次の問題設定を考えます。

問題設定

* エピソードの数：$K$
* $k$回目の方策：$\pi^k=\left\{\pi_0^k, \ldots, \pi_{H-1}^k\right\}$
* ホライズン：$H$
* 軌跡：$\tau^k=\left\{s_h^k, a_h^k\right\}_{h=0}^{H-1}$
* 行動：$a_h \sim \pi_h^k (s_h^k)$
* 遷移確率：$s_{h+1}^k \sim P^\star_h\left(\cdot \mid s_h^k, a_h^k\right)$
* 次状態遷移の回数：$N_h^k\left(s, a, s^{\prime}\right)=\sum_{i=0}^{k-1} \mathbf{1}\left\{\left(s_h^i, a_h^i, s_{h+1}^i\right)=\left(s, a, s^{\prime}\right)\right\}$
* 状態行動の訪問回数：$N_h^k(s, a)=\sum_{i=0}^{k-1} \mathbf{1}\left\{\left(s_h^i, a_h^i\right)=(s, a)\right\}, \forall h, s, a$.
* 遷移確率の推定：$\widehat{P}_h^k\left(s^{\prime} \mid s, a\right)=\frac{N_h^k\left(s, a, s^{\prime}\right)}{N_h^k(s, a)}, \forall h, s, a, s^{\prime}$.
* (偽)リグレット：$\sum_{k=0}^{K-1}\left(V^{\star}\left(s_0\right)-V^{\pi^k}\left(s_0\right)\right)$

### UCB-VIアルゴリズムの証明

\begin{aligned}
& \text { Algorithm } 5 \text { UCBVI } \\
& \text { Input: reward function } r \text { (assumed to be known), confidence parameters } \\
& \text { 1: for } k=0 \ldots K-1 \text { do } \\
& \text { 2: } \quad \text { Compute } \widehat{P}_h^k \text { as the empirical estimates, for all } h \\
& \text { 3: } \quad \text { Compute reward bonus } b_h^k \text { for all } \mathrm{h} \\
& \text { 4: } \quad \text { Run Value-Iteration on }\left\{\widehat{P}_h^k, r+b_h^k\right\}_{h=0}^{H-1} \\
& \text { 5: } \quad \text { Set } \pi^k \text { as the returned policy of VI. } \\
& \text { 6: end for }
\end{aligned}

さらにアルゴリズム特有の記法を導入します。

* $L:=\ln (S A H K / \delta)$
* ボーナス項：$b_h^k(s, a)=2 H \sqrt{\frac{L}{N_h^k(s, a)}}$
* 価値反復法：

\begin{equation}
\begin{aligned}
& \widehat{V}_H^k(s)=0, \forall s \\
& \widehat{Q}_h^k(s, a)=\min \left\{r_h(s, a)+b_h^k(s, a)+\widehat{P}_h^k(\cdot \mid s, a) \cdot \widehat{V}_{h+1}^k, H\right\} \\
& \widehat{V}_h^k(s)=\max _a \widehat{Q}_h^k(s, a), \pi_h^k(s)=\operatorname{argmax}_a \widehat{Q}_h^k(s, a), \forall h, s, a .
\end{aligned}
\end{equation}

これはモデルベースなアルゴリズムですね。
このアルゴリズムに対してリグレットの上界を証明していきます。

参考
* [Reinforcement Learning: Theory and Algorithms](https://rltheorybook.github.io/)



#### 1. 遷移確率の推定 (弱め)

推定した遷移確率の誤差のバウンドを確認しましょう。
つまり、任意の関数$f:\mathcal{S} \mapsto [0, H]$について、$\left|\left(\widehat{P}_h^k(\cdot \mid s, a)-P_h^{\star}(\cdot \mid s, a)\right)^{\top} f\right|$の上界を導出します。
まず定義より、

$$
\widehat{P}_h^k(\cdot \mid s, a)^{\top} f - {P}_h^\star(\cdot \mid s, a)^{\top} f =
\frac{1}{N_h^k(s, a)}
\sum_{i=0}^{k-1} \left(\mathbf{1}\left\{s_h^i, a_h^i=s, a\right\} f\left(s_{h+1}^i\right)
- \mathbf{1}\left\{s_h^i, a_h^i=s, a\right\} \mathbb{E}_{s^{\prime} \sim P_h^{\star}(s, a)} f\left(s^{\prime}\right)\right)
$$
です。


ここで、和の中身について、$X_i=\mathbf{1}\left\{s_h^i, a_h^i=s, a\right\} f\left(s_{h+1}^i\right)-\mathbf{1}\left\{s_h^i, a_h^i=s, a\right\} \mathbb{E}_{s^{\prime} \sim P_h^*}(s, a) f\left(s^{\prime}\right)$とします。
また、$\mathcal{H}_{h, <i}$を$h$での$i$を含まない$i$までの履歴とすると、決定的なアルゴリズムについては$\mathbb{E}\left[X_i \mid \mathcal{H}_{h, <i}\right]=0$なので、$\sum_{i=0}^{k-1} X_i$はマルチンゲールです。
また、$\left(s_h^i, a_h^i\right) \neq(s, a)$では$\left|X_i\right|=0$, $\left(s_h^i, a_h^i\right)=(s, a)$では$\left|X_i\right| \leq 2H$であることに気をつければ、
次のAzuma-Hoeffdingのバウンドが使えます：

---
**定理：Azuma-Hoeffdingの不等式**

$\left(\mathcal{F}_n\right)_{n=1}^N$に対して可測な実数値確率過程$\left(X_n\right)_{n=1}^N$を考える. 
$X_n \in\left[l_n, u_n\right]$ かつ $\mathbb{E}_n\left[X_n\right]=0$ が任意の$n$でa.s.で成立するとする. 
このとき、
$$
P\left(\sum_{n=1}^N X_n \geq \sqrt{\sum_{n=1}^N \frac{\left(u_n-l_n\right)^2}{2} \log \frac{1}{\delta}}\right) \leq \delta
$$

---

これを使うと、固定された$a$, $s$, $h$, $k$, そして$f$について、$1-\delta$以上で次が成立します。
$$
\left|\sum_{i=0}^{k-1} X_i\right|=
\left|\sum_{i=0}^{k-1} \left(\mathbf{1}\left\{s_h^i, a_h^i=s, a\right\} f\left(s_{h+1}^i\right)
- \mathbf{1}\left\{s_h^i, a_h^i=s, a\right\} \mathbb{E}_{s^{\prime} \sim P_h^{\star}(s, a)} f\left(s^{\prime}\right)\right)\right|
\leq \square H \sqrt{N_h^k(s, a) \ln (1 / \delta)}
$$

そして$s \in \mathcal{S}, a \in \mathcal{A}, h \in[0, \ldots, H-1], k \in[0, \ldots, K-1]$にUnion boundを取れば、任意の$s, a, h, k$について
$$
P\left(\left|\widehat{P}_h^k(\cdot \mid s, a)^{\top} f - {P}_h^\star(\cdot \mid s, a)^{\top} f\right|\geq \square H \sqrt{\frac{\ln (SAKH / \delta)}{N_h^k(s, a)}}\right) \leq {\delta}
$$
です。

ここでまでの話では、まだ$f$についてのUnion boundを取っていません。バンディットの場合は$f$が報酬関数で固定されていたのでUnion Boundを取らなくても良かったわけですが、今回は$f$が$f:\mathcal{S} \mapsto [0, H]$で動き回ります。
一方で、この$f$は全部で$\infty$個存在するので、単純にUnion Boundを取ることができません。そこで、$\epsilon$-net法を使います。詳しくは[Covering with $\varepsilon$-nets](https://erdogdu.github.io/csc2532/lectures/lecture05.pdf)参照。

まず任意の$f: \mathcal{S} \mapsto[0, H]$について、$\|f\|_2 \leq H \sqrt{S}$であることに注意しましょう。
詳細は[Covering with $\varepsilon$-nets](https://erdogdu.github.io/csc2532/lectures/lecture05.pdf)に任せますが、このときどんな$f\in[0, H]^S$に対しても、$\left\|f-f^{\prime}\right\|_2 \leq \epsilon$を満たすような$f'\in \mathcal{N}_\varepsilon$が存在できる$\mathcal{N}_\varepsilon$のサイズは$\left|\mathcal{N}_\epsilon\right| \leq(1+2 H \sqrt{S} / \epsilon)^S$になります (リンク先のFigure2が分かりやすいです)。この$\mathcal{N}_\varepsilon$のサイズを``Covering Number``と呼びます。

これを使って$s, a, k, h, f \in \mathcal{N}_\varepsilon$についてUnion Boundを取ると、

$$
P\left(\left|\widehat{P}_h^k(\cdot \mid s, a)^{\top} f' - {P}_h^\star(\cdot \mid s, a)^{\top} f'\right|\geq \square H \sqrt{\frac{\ln (SAKH(1+2 H \sqrt{S} / \epsilon) / \delta)}{N_h^k(s, a)}}\right) \leq {\delta}
$$

最後に任意の$f \in[0, H]^S$についてのバウンドに変形します。次のバウンドが確率$1-\delta$で成立します。

$$
\begin{aligned}
\left|\widehat{P}_h^k(\cdot \mid s, a)^{\top} f - {P}_h^\star(\cdot \mid s, a)^{\top} f\right|
&=\left|
\widehat{P}_h^k(\cdot \mid s, a)^{\top} f' - {P}_h^\star(\cdot \mid s, a)^{\top} f'
+ \widehat{P}_h^k(\cdot \mid s, a)^{\top}(f - f')
+ \widehat{P}_h^\star(\cdot \mid s, a)^{\top}(f' - f)
\right|\\
&\leq\left|\widehat{P}_h^k(\cdot \mid s, a)^{\top} f' - {P}_h^\star(\cdot \mid s, a)^{\top} f'\right|
+ \left|\widehat{P}_h^k(\cdot \mid s, a)^{\top}(f - f')\right|
+ \left|\widehat{P}_h^\star(\cdot \mid s, a)^{\top}(f' - f)\right|\\
&\leq \square H \sqrt{\frac{\ln (SAKH(1+2 H \sqrt{S} / \varepsilon) / \delta)}{N_h^k(s, a)}} + 2\varepsilon
\end{aligned}
$$

最後の不等式では$\|f-f^{\prime} \|_2 \leq \epsilon$ならば$\left|f(s)-f^{\prime}(s)\right| \leq \epsilon, \forall s \in \mathcal{S}$であることを使いました。([Useful Inequalities Cheat Sheet](https://www.lkozma.net/inequalities_cheat_sheet/ineq.pdf)のMean参照)

あとは$\varepsilon$を決めるだけです。
よく見ると$\varepsilon$は$\ln$の中で登場していますね。つまり、我々が対数の中身を気にしない限り、$\varepsilon$は多項式であれば何でも良いわけですね。
また、$N_h^k(s, a) \leq K$と上のバウンドで$1\varepsilon$が出てきていることも踏まえると、$\varepsilon=1/K$として良いでしょう。([Notes on Exploration in Linear MDPs](https://nanjiang.cs.illinois.edu/files/cs542f22/note8.pdf)を参考にしましたが、やや自信ないです。)

以上をまとめると、


$$
\left|\widehat{P}_h^k(\cdot \mid s, a)^{\top} f - {P}_h^\star(\cdot \mid s, a)^{\top} f\right|
\leq \square H \sqrt{\frac{S \ln (S A K H(1+2 H K \sqrt{S}) / \delta)}{N_h^k(s, a)}}+\frac{\square}{K}
\leq \square H \sqrt{\frac{S \ln (H S K A / \delta)}{N_h^k(s, a)}},
$$

になり、これが欲しかったバウンドになります。

実はこれはそんなに良いバウンドではありません。が、Covering Numberは大事な概念なので載せておきました。

#### 2. 遷移確率＋$V^\star$の推定

ここでは$V^\star$が固定されていることを利用すれば、1.の議論からすぐに次の結果が得られます。
任意の$s, a, h, k$について
$$
P\left(\left|\widehat{P}_h^k(\cdot \mid s, a)^{\top} V_{h+1}^\star - {P}_h^\star(\cdot \mid s, a)^{\top} V^\star_{h+1}\right|\geq \square H \sqrt{\frac{\ln (SAKH / \delta)}{N_h^k(s, a)}}\right) \leq {\delta}
$$
が成り立ちます。

#### 3. $V^{\star}\left(s_0\right) \leq \widehat{V}^{k}\left(s_0\right)$の証明

* 条件：1.と2.のバウンドが成り立っているとき

リグレットをバウンドするために、UCBアルゴリズムと似たようなことをしていきます。

リグレットは
$\sum_{k=0}^{K-1}\left(V^{\star}\left(s_0\right)-V^{\pi^k}\left(s_0\right)\right)$だったので、
$V^{\star}\left(s_0\right)-V^{\pi^k}\left(s_0\right)$がバウンドできればほぼゴールです。

これをバウンドするためには、UCBのときと似た感じで
$V^{\star}\left(s_0\right)-V^{\pi^k}\left(s_0\right) \leq \widehat{V}^{k}\left(s_0\right)-V^{\pi^k}\left(s_0\right)$
が成り立つことを利用したいわけですね。

これは再帰的に確認できます。まず、$h=H$については
$\widehat{V}^k_H(s) = \widehat{V}^\star_H(s)=0$なのでOKです。$h+1$のときにOKだとすると、任意の$(s, a)\in \mathcal{S}\times \mathcal{A}$について、

$$
\begin{aligned}
\widehat{Q}_h^k(s, a)-Q_h^{\star}(s, a) & =b_h^k(s, a)+\widehat{P}_h^k(\cdot \mid s, a) \cdot \widehat{V}_{h+1}^k-P_h^{\star}(\cdot \mid s, a) \cdot V_{h+1}^{\star} \\
& \geq b_h^k(s, a)+\widehat{P}_h^k(\cdot \mid s, a) \cdot V_{h+1}^{\star}-P_h^{\star}(\cdot \mid s, a) \cdot V_{h+1}^{\star} \\
& =b_h^k(s, a)+\left(\widehat{P}_h^k(\cdot \mid s, a)-P_h^{\star}(\cdot \mid s, a)\right) \cdot V_{h+1}^{\star} \\
& \geq b_h^k(s, a)-\square H \sqrt{\frac{\ln (S A H K / \delta)}{N_h^k(s, a)}} \geq 0 .
\end{aligned}
$$

が成立します。

#### 4. $\sum_{k=0}^{K-1} \sum_{h=0}^{H-1} \frac{1}{\sqrt{N_h^k\left(s_h^k, a_h^k\right)}}$のバウンド

遷移確率のバウンドで出てきた$\frac{1}{\sqrt{N_h^k(s, a)}}$は後々バウンドしないといけません。今のうちにバウンドしておきましょう。

$$
\begin{aligned}
& \sum_{k=0}^{K-1} \sum_{h=0}^{H-1} \frac{1}{\sqrt{N_h^k\left(s_h^k, a_h^k\right)}}=\sum_{h=0}^{H-1} \sum_{k=0}^{K-1} \frac{1}{\sqrt{N_h^k\left(s_h^k, a_h^k\right)}}=\sum_{h=0}^{H-1} \sum_{s, a \in \mathcal{S} \times \mathcal{A}} \sum_{i=1}^{N_h^K(s, a)} \frac{1}{\sqrt{i}} \\
& \leq 2 \sum_{h=0}^{H-1} \sum_{s, a \in \mathcal{S} \times \mathcal{A}} \sqrt{N_h^K(s, a)} \leq \sum_{h=0}^{H-1} \sqrt{S A \sum_{s, a} N_h^K(s, a)}=H \sqrt{S A K},
\end{aligned}
$$

１つ目の不等式は[Upper Bound of $\sum_{k=1}^{n} \frac{1}{\sqrt{k}}$](https://math.stackexchange.com/questions/3745324/upper-bound-of-sum-k-1n-frac1-sqrtk)参考。２つ目の不等式はコーシー・シュワルツか平均の不等式を使います。[Useful Inequalities Cheat Sheet](https://www.lkozma.net/inequalities_cheat_sheet/ineq.pdf)参考。

#### 5. リグレットのバウンド

* 条件：1.と2.のバウンドが成り立っているとき

ここまで来たらリグレットをバウンドしていきます。

1と2で求めたバウンドが成り立っている事象を考えているので、3の結果を使うと、

$$
V^{\star}\left(s_0\right)-V^{\pi^k}\left(s_0\right) \leq \widehat{V}_0^k\left(s_0\right)-V_0^{\pi^k}\left(s_0\right)
$$

が成り立っています。
